# General Idea of a Deep Neural Network
![](dnn.png)

In [ ]:
# set up our model
import tensorflow as tf

# x_train is our training images/data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# this normalizes our picture
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

# relu (rectified linear unit) is f(x) = 0 for x < 0 and f(x) = x for x >= 0
# softmax function transforms our vector of real values into a vector of real values that sum to 1 (so we can use them as probabilities)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()) # our input layer
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu)) # first hidden layer
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu)) # second hidden layer
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax)) # output layer, 10 is how many possibilities there are (0-9 in our case)

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3)

In [ ]:
# lets see our actual loss and accuracy using our testing data
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)
# expected to be slightly different from what you get from above (loss higher, accuracy lower) but you don't want it to be too close or too far

In [ ]:
# show some of our training data
import matplotlib.pyplot as plt
plt.imshow(x_train[0], cmap=plt.cm.binary)
plt.show()

In [ ]:
# save our model as 'num_reader.model'
model.save('num_reader.model')

In [ ]:
# load our model
new_model = tf.keras.models.load_model('num_reader.model')

In [ ]:
# predict an image
predictions = new_model.predict(x_test)

In [ ]:
# these are one-hot arrays
print(predictions)

In [ ]:
# we can use numpy to get the above into something a bit more readable
import numpy as np
print(np.argmax(predictions[0])) # this prints the highest confidence guess

In [ ]:
# lets verify that the image is 7
plt.imshow(x_test[0], cmap=plt.cm.binary)
plt.show()

In [ ]:
# lets see some examples of numbers 0-9
num = 0
while num < 10:
    for i, p in enumerate(predictions):
        if np.argmax(p) == num:
            plt.imshow(x_test[i], cmap=plt.cm.binary)
            plt.show()
            num += 1
            break


In [ ]:
# now lets make some of our own pictures and see if our model can recognize them
# when we print out our model's guess lets also print out the probabilities of each number
# np.set_printoptions(suppress=True) # this disables scientific notation when we print out probabilities
# some very small probabilities might not print correctly with this on
file_names = ['four1.png', 'two1.png', 'zero1.png', 'four2.png', 'two2.png', 'zero2.png']
for fname in file_names:
    img = tf.keras.preprocessing.image.load_img(fname, color_mode='grayscale')
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    # x = tf.image.resize(x, (28, 28), antialias=True)
    x = tf.keras.utils.normalize(x, axis=1)
    prediction = new_model.predict(x)
    print(np.argmax(prediction))
    print(prediction)

In [ ]:
# now lets compare our output to our pictures above to check if its correct
import matplotlib.image as mpimg

for fname in file_names:
    image = mpimg.imread(fname)
    plt.imshow(image)
    plt.show()

# Concluding Information
As you can see our number outputs match the pictures; these pictures were made in paint, however if you draw on paper and pencil and try and have the model predict each number it will likely fail.
## some properties you need to apply (preprocess) in most cases:
1. white numbers, black background
2. 28x28 bounding box, but the number is normalized to fix in a 20x20 box
3. Centered by center of mass

In our case since our examples comply to #1, and somewhat comply to #2 and #3 our model can suprisingly recognize them
however it still likely have trouble since we did not actually apply any actual preprocessing. In fact, if our images were moved more off center
it would be very likely that the model would predict a different number.